In [31]:
!pip install dash

In [2]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
spacex_df=pd.read_csv("/content/spacex_launch_dash.csv")

In [6]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [7]:
# Create a dash application
app = dash.Dash(__name__)

In [18]:
#create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),])

In [20]:
# TASK 1: Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites
dcc.Dropdown(id='site-dropdown',options=[
{'label': 'All Sites', 'value': 'ALL'},
{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
{'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
{'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
{'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
value='ALL',
placeholder="Select a Launch Site here",
searchable=True),html.Br(), html.Br(),

# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
html.Div(dcc.Graph(id='success-pie-chart')),
html.Br(),
html.P("Payload range (Kg):"),
# TASK 3: Add a slider to select payload range
#dcc.RangeSlider(id='payload-slider',...)
dcc.RangeSlider(
id='payload-slider',
min=0,
max=10000,
step=1000,
marks={0: '0',100: '100'},
value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]),
# TASK 4: Add a scatter chart to show the correlation between payload and launch success
html.Div([dcc.Graph(id='success-payload-scatter-chart'),])
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes pie chart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

        # Calculate success and failure count for the selected site
        success_count = filtered_df[filtered_df['class'] == 1].shape[0]
        failure_count = filtered_df[filtered_df['class'] == 0].shape[0]

        # Create pie chart for the selected site
        fig = px.pie(filtered_df,
            values=[success_count, failure_count],
            names=['Success', 'Failure'],
            title=f'Success and Failure Launches for {entered_site}')
        return fig
        # TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value"))
def update_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df.copy()

    # Step 1: Filter DataFrame based on selected site
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]

    # Step 2: Filter DataFrame based on payload range
    filtered_df = filtered_df[
        (filtered_df['Payload Mass (kg)'] >= payload_range[0]) &
        (filtered_df['Payload Mass (kg)'] <= payload_range[1])
    ]

    # Create and update the scatter plot
    fig = px.scatter(
        filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title=f'Correlation Between Payload and Success for {entered_site}', labels={'class': 'Class'}
    )
    return fig

In [20]:
# Run the app
if __name__ == '__main__':
    app.run_server()